In [0]:
import pandas as pd
import numpy as np
from tqdm import trange
from tqdm import tqdm_notebook as tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
data = pd.read_csv("/content/drive/My Drive/ner_dataset.csv", encoding="latin1").fillna(method="ffill")
data.tail(10)

,Sentence #,Word,POS,Tag
1048565,Sentence: 47958,impact,NN,O
1048566,Sentence: 47958,.,.,O
1048567,Sentence: 47959,Indian,JJ,B-gpe
1048568,Sentence: 47959,forces,NNS,O
1048569,Sentence: 47959,said,VBD,O
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O
1048574,Sentence: 47959,attack,NN,O


In [0]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [0]:
getter = SentenceGetter(data)

In [6]:
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'

In [0]:
# labels = [[s[2] for s in sent] for sent in getter.sentences]
# print(labels[0])

In [0]:
tags_vals = sorted(list(set(data["Tag"].values)))
tag2idx = {t: i for i, t in enumerate(tags_vals)}

In [9]:
tag2idx

{'B-art': 0,
 'B-eve': 1,
 'B-geo': 2,
 'B-gpe': 3,
 'B-nat': 4,
 'B-org': 5,
 'B-per': 6,
 'B-tim': 7,
 'I-art': 8,
 'I-eve': 9,
 'I-geo': 10,
 'I-gpe': 11,
 'I-nat': 12,
 'I-org': 13,
 'I-per': 14,
 'I-tim': 15,
 'O': 16}

In [10]:
# !pip uninstall pytorch-pretrained-bert
!pip install pytorch-pretrained-bert==0.4.0

     |████████████████████████████████| 51kB 5.7MB/s 


In [11]:
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

Using TensorFlow backend.


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [0]:
MAX_LEN = 75
bs      = 32

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [14]:
device

device(type='cuda')

In [15]:
if "cuda" in str(device) : 
  print(torch.cuda.get_device_name(0))

Tesla P4


In [16]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

100%|██████████| 231508/231508 [00:00<00:00, 1291236.39B/s]


In [17]:
sentences[0]

'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'

In [20]:
tokenized_texts = []
labels          = []
for sentence in tqdm(getter.sentences) :
    # Split into tokens by spaces
    # Now split each token into sub-tokens using the tokenizer
    # such that any new sub-tokens receive either a "O" or "I-"
    # label as necessary
    words  = []
    lls    = []
    for i in range(len(sentence)) :
        # Not sure why there's some whitespaces but OK
        word  = tokenizer.tokenize(sentence[i][0])
        if len(word) == 0 :
            continue
        label = [sentence[i][2]]
        if len(word)>1 :
            label.extend([label[0].replace("B-","I-")]*(len(word)-1))
#             print(word,label)
#             input()
        try : 
            assert(len(word)==len(label))
        except : 
            print("+"+sentence[i][0]+"+")
            print(word,label)
            raise Exception()
        words.extend(word)
        lls.extend(label)
    tokenized_texts.append(words)
    labels.append(lls)

In [0]:
# tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
assert(len(tokenized_texts[6])==len(labels[6]))

In [0]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [0]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")

In [0]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [0]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [0]:
tr_inputs  = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags    = torch.tensor(tr_tags)
val_tags   = torch.tensor(val_tags)
tr_masks   = torch.tensor(tr_masks)
val_masks  = torch.tensor(val_masks)

In [0]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [30]:
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(tag2idx))

100%|██████████| 407873900/407873900 [00:09<00:00, 43204463.32B/s]


In [0]:
if "cuda" in str(device) : 
  model.cuda();

In [0]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [33]:
!pip install seqeval

  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=98732b9b4cca6899305544e9ec2e994e1aaa16de56b6a784f5b8a640cccb12ad
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [0]:
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
from datetime import datetime
from google.colab import output

In [0]:
niter = len(list(enumerate(train_dataloader)))

In [37]:
epochs = 5
max_grad_norm = 1.0

dstart=datetime.now()
for iepoch in range(epochs) : # trange(epochs, desc="Epoch"):
    print("Working on epoch %i"%(iepoch))
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    ientry=0
    tepoch_start = datetime.now()
    for step, batch in enumerate(train_dataloader) :
        ientry+=1
        if ientry%50==0 :
          tnow    = datetime.now()
          remtime = ((tnow - tepoch_start)/ientry)*(niter-ientry)
          print("Epoch %i : Processing %i/%i entries, elapsed time: %s, remaining time in epoch: %s"%(iepoch,ientry,niter,datetime.now()-dstart,remtime))
#          output.clear('status_text')
#          with output.use_tags('status_text') :
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))

    # torch.save(model, "/content/drive/My Drive/ner.dataset.%i.pth"%(iepoch))

    # See: https://pytorch.org/tutorials/beginner/saving_loading_models.html
    torch.save(model.state_dict(), "/content/drive/My Drive/ner.dataset.%i.pth"%(iepoch))

    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
    print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

Working on epoch 0
Epoch 0 : Processing 50/1349 entries, elapsed time: 0:00:29.895882, remaining time in epoch: 0:12:56.651316
Epoch 0 : Processing 100/1349 entries, elapsed time: 0:01:00.103555, remaining time in epoch: 0:12:30.672731
Epoch 0 : Processing 150/1349 entries, elapsed time: 0:01:30.413748, remaining time in epoch: 0:12:02.693653
Epoch 0 : Processing 200/1349 entries, elapsed time: 0:02:00.891458, remaining time in epoch: 0:11:34.511901
Epoch 0 : Processing 250/1349 entries, elapsed time: 0:02:31.610624, remaining time in epoch: 0:11:06.473164
Epoch 0 : Processing 300/1349 entries, elapsed time: 0:03:02.746808, remaining time in epoch: 0:10:38.999399
Epoch 0 : Processing 350/1349 entries, elapsed time: 0:03:34.043771, remaining time in epoch: 0:10:10.937451
Epoch 0 : Processing 400/1349 entries, elapsed time: 0:04:05.212396, remaining time in epoch: 0:09:41.762623
Epoch 0 : Processing 450/1349 entries, elapsed time: 0:04:36.509294, remaining time in epoch: 0:09:12.400439
E

In [0]:
# !ls -ld "/content/drive/My Drive/ner.dataset.1.pth"

In [0]:
# Reload the model
# model = TheModelClass(*args, **kwargs)
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(tag2idx))
model.load_state_dict(torch.load("/content/drive/My Drive/ner.dataset.4.pth"))
model.eval()
model.cuda()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [0]:
for i in range(len(b_input_ids)) : 
  input_ids       = torch.tensor([b_input_ids[i].cpu().numpy()]).to(device)
  tags            = torch.tensor([b_labels[i].cpu().numpy()]).to(device)
  attention_masks = torch.tensor([b_input_mask[i].cpu().numpy()]).to(device)

  outputs   = model(input_ids, token_type_ids=None,
                      attention_mask=attention_masks) # , labels=tags)

  ovar = outputs.to('cpu').detach().numpy()

  pred_labels = [np.argmax(ovar[0][j]) for j in range(len(ovar[0]))]
  if np.sum(pred_labels) != 440 :
    print(pred_labels)

  # loss, scores = outputs[:2]